<a href="https://colab.research.google.com/github/LucasLessa1/X-rayBodyPart/blob/davi1/Proj_CIS_Imagens_Raio_X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports and Ajusting file


In [ ]:
pip install pydicom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 6.1 MB/s 


In [98]:
import cv2
import pydicom
import pandas as pd
import os
import shutil
import time
import math
import gdown
import zipfile
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from tqdm import tqdm
import os
from PIL import Image
from google.colab.patches import cv2_imshow
import string

In [92]:
def download(id): 
    url = 'https://drive.google.com/uc?id=' + str(id)
    gdown.download(url, output = None, quiet = False)

def unzip(path): #Função para unzip
    zip = zipfile.ZipFile(path)
    zip.extractall()
    zip.close()

In [ ]:
# https://drive.google.com/file/d/1ev-r31j8oRzDlKM_toaeADO2psrA_XXm/view?usp=sharing

download('1ev-r31j8oRzDlKM_toaeADO2psrA_XXm')
unzip('/content/archive.zip')

Downloading...
From: https://drive.google.com/uc?id=1ev-r31j8oRzDlKM_toaeADO2psrA_XXm
To: /content/archive.zip
100%|██████████| 269M/269M [00:01<00:00, 254MB/s]


In [111]:
os.chdir("/content/drive/MyDrive/RAIO-X/RAIO-X/archive.zip (Unzipped Files)")
path = os.getcwd() 
train_df = pd.read_csv(os.path.join(os.getcwd(), 'train.csv'))
test_df = pd.read_csv(os.path.join(os.getcwd(),'sample_submission.csv'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [100]:
test_df.head(5)

,SOPInstanceUID,Target
0,1.2.826.0.1.3680043.8.498.10001001190452685542...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
1,1.2.826.0.1.3680043.8.498.10022667601042710442...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
2,1.2.826.0.1.3680043.8.498.10024395388921105474...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
3,1.2.826.0.1.3680043.8.498.10026689165626095651...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
4,1.2.826.0.1.3680043.8.498.10035936364561920980...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...


In [101]:
A=train_df.head(1)['SOPInstanceUID']
print(A[0][50:])

00333618114258


#Pre-processing

In [102]:
print("Columns:")
print(train_df.columns, "\n")

print("Types in columns:")
print(train_df.dtypes, "\n")

print("Types in info:")
print(train_df.describe(), "\n")

Columns:
Index(['SOPInstanceUID', 'Target'], dtype='object') 

Types in columns:
SOPInstanceUID    object
Target            object
dtype: object 

Types in info:
                                           SOPInstanceUID Target
count                                                1738   1738
unique                                               1738     41
top     1.2.826.0.1.3680043.8.498.10025629581362719970...     3 
freq                                                    1    724 



##Fixing folders

In [233]:
class Image_processing():
  def __init__(self):
    self.fileList = []

  def fixfolders(self, path, df, dictionary):
    filelist = []
    for root, dirs, files in os.walk(os.path.join(path, "train")):
      for file in files:
        filelist.append(os.path.join(root,file))

    for filename in filelist:
      dicom = pydicom.dcmread(filename)
      img = dicom.pixel_array
      var = filename.split("/")
      var = var[-1]
      #patient = var[5]
      #code = var[6]
      row = df.index[df['SOPInstanceUID']==f'{var[:-6]}'].tolist()
      target = df['Target'].iloc[row[0]]
      if len(target.strip()) > 1: 
        continue
      label = dictionary.get(int(target))
      im = Image.fromarray(img.astype(np.uint8))
      im.save(os.path.join(path, "train_img", f'{label}', f"{var[:-6]}.png"))
      im.save(os.path.join(path, "test_img", f'{label}', f"{var[:-6]}.png"))
  def createfolders(self, folder):
    os.mkdir(os.path.join(path,folder)) 
    os.chdir(os.path.join(path,folder)) 
    for bodypart in list(bodyparts.values()):
      if os.path.isdir(f'{bodypart}') == False:
        body_path = os.path.join(os.getcwd(),f"{bodypart}")
        os.mkdir(body_path)
      else:  
          pass 

  def removeFolders(self, path): 
    #Para remover as pastas 
    os.chdir(os.path.join(path, "train"))
    path2 = os.getcwd()
    for folder in os.listdir(path2):  
      print(folder)
      if folder != "train":
        os.rmdir(os.path.join(path2, f'{folder}'))
      else: 
        pass 
    os.chdir(os.path.join(path, "test"))
    path2 = os.getcwd()
    for folder in os.listdir(path2):  
      if folder != "test": 
        os.rmdir(os.path.join(path2, f'{folder}'))
      else: 
       pass 



In [234]:
os.chdir("/content/drive/MyDrive/RAIO-X/RAIO-X/archive.zip (Unzipped Files)")
path = os.getcwd()
fix  =  Image_processing()
fix.fixfolders(path, train_df,bodyparts)

In [230]:
#EXCLUIR ARQUIVOS DENTRO DAS PASTARS LABELS 
import glob
os.chdir("/content/drive/MyDrive/RAIO-X/RAIO-X/archive.zip (Unzipped Files)")
for file in glob.glob('train_img/**/*.bmp'):
  print(file)
  os.remove(file)
os.chdir("/content/drive/MyDrive/RAIO-X/RAIO-X/archive.zip (Unzipped Files)")
for file in glob.glob('test_img/**/*.bmp'):
  print(file)
  os.remove(file)
# shutil.rmtree("/content/trainRepaired")
# shutil.rmtree("/content/trainRepaired2")
# shutil.rmtree("/content/testRepaired")

In [ ]:
try:
  image_processing = Image_processing("train")
  image_processing.fixFolders("trainRepaired")

  image_processing2 = Image_processing("test")
  image_processing2.fixFolders("testRepaired")
except FileExistsError:
  shutil.rmtree("/content/" + "trainRepaired")
  shutil.rmtree("/content/" + "testRepaired" )

  image_processing = Image_processing("train")
  image_processing.fixFolders("trainRepaired")

  image_processing2 = Image_processing("test")
  image_processing2.fixFolders("testRepaired")

In [ ]:
train_df

,SOPInstanceUID,Target
0,1.2.826.0.1.3680043.8.498.10025629581362719970...,0
1,1.2.826.0.1.3680043.8.498.10036150326276641158...,15
2,1.2.826.0.1.3680043.8.498.10038426859954986240...,12
3,1.2.826.0.1.3680043.8.498.10050991192143676483...,14
4,1.2.826.0.1.3680043.8.498.10053309524595490852...,3
...,...,...
1733,1.2.826.0.1.3680043.8.498.99711016467406172920...,14
1734,1.2.826.0.1.3680043.8.498.99727280485333900933...,11
1735,1.2.826.0.1.3680043.8.498.99935008698914755885...,3
1736,1.2.826.0.1.3680043.8.498.99969671777553755311...,2


In [ ]:
lista=[[[1]],[2], [3]]

In [ ]:
def dcmtag2table(folder, list_of_tags):

    list_of_tags = list_of_tags.copy()
    items = []
    table = []
    filelist = []

    for root, dirs, files in os.walk(folder, topdown=False):
#Adiciona os caminhos de cada imagem na lista
        for name in files:
            filelist.append(os.path.join(root, name))
    aux= 0
    for _f in filelist:
      #Em cada caminho da imagem(.dcm)
        try:
            ds = pydicom.dcmread(_f, stop_before_pixels=True)
            #Pega o Dataset de cada imagem e ler no próximo for
            items = []
            items.append(_f)
            #Adiciono o caminho em "items"
            for _tag in list_of_tags:
                if _tag in ds:
                  
                    items.append(ds.data_element(_tag).value)
                    #Adiciono a features "PhotometricInterpretation", "BitsAllocated" and "SOPInstanceUID" em "items"
                else:
                    items.append("Not found")
            
            table.append(items)
            #Adiciona tudo em uma outra lista
        except:
            print("Skipping non-DICOM: " + _f)

    df = pd.DataFrame(table, columns=["Filename", "PhotometricInterpretation", "BitsAllocated","SOPInstanceUID"])
    #Crio um data set com todos os caminhos Photometric, Bits and SOP
    return df

*   SOP Instance UID - Unique Identification
*   SOP Class UID    - 

In [ ]:
tags = ['PhotometricInterpretation','BitsAllocated', 'SOPInstanceUID' ]
dicom_tags_train =  dcmtag2table('/content/train', tags)
dicom_tags_test = dcmtag2table('/content/test', tags)

In [ ]:
dicom_tags_train

,Filename,PhotometricInterpretation,BitsAllocated,SOPInstanceUID
0,/content/train/train/train/133/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.77469401758687792239...
1,/content/train/train/train/133/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.86362988389115608778...
2,/content/train/train/train/367/1.2.826.0.1.368...,MONOCHROME2,8,1.2.826.0.1.3680043.8.498.71782455619707465800...
3,/content/train/train/train/367/1.2.826.0.1.368...,MONOCHROME2,8,1.2.826.0.1.3680043.8.498.58996250760352585732...
4,/content/train/train/train/1371/1.2.826.0.1.36...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.95649369759591173352...
...,...,...,...,...
1733,/content/train/train/train/1353/1.2.826.0.1.36...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.81821820223500601248...
1734,/content/train/train/train/687/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.13045859977087867316...
1735,/content/train/train/train/687/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.85277699722730898683...
1736,/content/train/train/train/730/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.95185429210875251933...


In [ ]:
print(dicom_tags_train.PhotometricInterpretation.value_counts())

MONOCHROME1    1659
MONOCHROME2      79
Name: PhotometricInterpretation, dtype: int64


In [ ]:
train = dicom_tags_train.merge(train_df, on =  'SOPInstanceUID')
test = dicom_tags_test.merge(test_df,on =  'SOPInstanceUID')

In [ ]:
train.head()

,Filename,PhotometricInterpretation,BitsAllocated,SOPInstanceUID,Target
0,/content/train/train/train/133/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.77469401758687792239...,3
1,/content/train/train/train/133/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.86362988389115608778...,3
2,/content/train/train/train/367/1.2.826.0.1.368...,MONOCHROME2,8,1.2.826.0.1.3680043.8.498.71782455619707465800...,14
3,/content/train/train/train/367/1.2.826.0.1.368...,MONOCHROME2,8,1.2.826.0.1.3680043.8.498.58996250760352585732...,14
4,/content/train/train/train/1371/1.2.826.0.1.36...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.95649369759591173352...,9


In [ ]:
target = list(train["Target"])

In [187]:
#Let's  see our Target distribution

bodyparts = {
0 : 'Abdomen' ,
1 :'Ankle' ,
2 :'Cervical Spine',
3 : 'Chest' ,
4 :'Clavicles' ,
5 :'Elbow' ,
6 :'Feet' ,
7 : 'Finger' ,
8 : 'Forearm' ,
9 : 'Hand' ,
10 : 'Hip' ,
11 : 'Knee' ,
12 : 'Lower Leg' ,
13 : 'Lumbar Spine' ,
14 : 'Others' ,
15 :'Pelvis',
16 :'Shoulder' ,
17 :'Sinus' ,
18 : 'Skull' ,
19 : 'Thigh' ,
20 :'Thoracic Spine',
21: 'Wrist',
}

In [ ]:
labels = ['Abdomen', 'Ankle', 'Cervical Spine',
       'Chest', 'Clavicles', 'Elbow', 'Feet', 'Finger', 'Forearm', 'Hand',
       'Hip', 'Knee', 'Lower Leg', 'Lumbar Spine', 'Others', 'Pelvis',
       'Shoulder', 'Sinus', 'Skull', 'Thigh', 'Thoracic Spine', 'Wrist']

In [ ]:
#Create Label from target
def no_to_label(label):
    label_list_string = []
    trimed_label = label.rstrip()
    label_list = trimed_label.split(" ")
    label_list = [int(i) for i in label_list]
    for label in label_list:
        label_list_string.append(bodyparts[label])
    label_string = ' and '.join(label_list_string)
    return label_string

target_list = train['Target'].tolist()
label_column = []

for label in tqdm(target_list):
    label_string = no_to_label(label)
    label_column.append(label_string)

train['Label'] = label_column
train['Label'].unique()

100%|██████████| 1738/1738 [00:00<00:00, 364157.28it/s]


array(['Chest', 'Others', 'Hand', 'Sinus',
       'Cervical Spine and Lumbar Spine and Thoracic Spine',
       'Lumbar Spine and Thoracic Spine', 'Pelvis', 'Wrist',
       'Cervical Spine', 'Abdomen', 'Lumbar Spine', 'Clavicles',
       'Thoracic Spine', 'Lower Leg', 'Feet', 'Finger', 'Knee',
       'Ankle and Feet', 'Ankle', 'Hip', 'Hand and Wrist',
       'Abdomen and Chest', 'Skull', 'Forearm', 'Elbow', 'Shoulder',
       'Thigh', 'Hip and Thigh', 'Ankle and Knee and Lower Leg',
       'Pelvis and Thigh', 'Knee and Lower Leg', 'Forearm and Shoulder',
       'Elbow and Forearm', 'Chest and Pelvis', 'Chest and Feet',
       'Ankle and Lower Leg', 'Abdomen and Pelvis', 'Chest and Finger',
       'Hip and Knee and Thigh', 'Cervical Spine and Thoracic Spine',
       'Finger and Hand'], dtype=object)

In [ ]:
train

,Filename,PhotometricInterpretation,BitsAllocated,SOPInstanceUID,Target,Label
0,/content/train/train/train/133/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.77469401758687792239...,3,Chest
1,/content/train/train/train/133/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.86362988389115608778...,3,Chest
2,/content/train/train/train/367/1.2.826.0.1.368...,MONOCHROME2,8,1.2.826.0.1.3680043.8.498.71782455619707465800...,14,Others
3,/content/train/train/train/367/1.2.826.0.1.368...,MONOCHROME2,8,1.2.826.0.1.3680043.8.498.58996250760352585732...,14,Others
4,/content/train/train/train/1371/1.2.826.0.1.36...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.95649369759591173352...,9,Hand
...,...,...,...,...,...,...
1733,/content/train/train/train/1353/1.2.826.0.1.36...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.81821820223500601248...,11,Knee
1734,/content/train/train/train/687/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.13045859977087867316...,3,Chest
1735,/content/train/train/train/687/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.85277699722730898683...,3,Chest
1736,/content/train/train/train/730/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.95185429210875251933...,13 20,Lumbar Spine and Thoracic Spine


In [ ]:
train.Label.value_counts()

Chest                                                 724
Others                                                120
Knee                                                  102
Abdomen                                                80
Cervical Spine                                         77
Feet                                                   70
Pelvis                                                 67
Wrist                                                  63
Lumbar Spine and Thoracic Spine                        45
Hand and Wrist                                         42
Ankle                                                  41
Shoulder                                               40
Lumbar Spine                                           39
Hand                                                   31
Sinus                                                  23
Elbow                                                  23
Lower Leg                                              19
Hip           